In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.transform import Rotation as R

fps = 240

def angular_velocities(q1, q2, dt):
    return (2 / dt) * np.array([
        q1[0]*q2[1] - q1[1]*q2[0] - q1[2]*q2[3] + q1[3]*q2[2],
        q1[0]*q2[2] + q1[1]*q2[3] - q1[2]*q2[0] - q1[3]*q2[1],
        q1[0]*q2[3] - q1[1]*q2[2] + q1[2]*q2[1] - q1[3]*q2[0]])

df = pd.read_csv('./scottie_240_frames/scottie240merged_data.csv')
unique_ftids = df['ftid'].unique()
backspin_df = pd.DataFrame(columns=['ftid', 'avg_backspin'])

for ftid in unique_ftids:
    df2 = df[df['ftid'] == ftid].copy()
    t_array = (1/fps * (df2['framesequence'] - df2['framesequence'].iloc[0])).to_numpy()
    print(ftid)

    x_pos_array = df2['ballx'].to_numpy() / 12
    y_pos_array = df2['bally'].to_numpy() / 12
    z_pos_array = df2['ballz'].to_numpy() / 12
    # x and y switch since 240 fps tracking data is flipped, might have to also flip x sign
    x_rot_array = df2['BALL_Rotation_Y'].to_numpy() 
    y_rot_array = df2['BALL_Rotation_X'].to_numpy() * -1
    z_rot_array = df2['BALL_Rotation_Z'].to_numpy()
    w_rot_array = df2['BALL_Rotation_W'].to_numpy()
    
    quaternions = np.vstack((w_rot_array, x_rot_array, y_rot_array, z_rot_array)).T
    
    # Compute angular velocities
    angvel = np.zeros((len(t_array), 3))
    for i in range(1, len(angvel)):
        dt = t_array[i] - t_array[i-1]
        angvel[i] = angular_velocities(quaternions[i-1], quaternions[i], dt)

    
    
    if x_pos_array[0] < 0:
        y_pos_array = -y_pos_array
        x_pos_array = -x_pos_array    
    
    v_x_array = np.gradient(x_pos_array, t_array, edge_order=1)
    v_y_array = np.gradient(y_pos_array, t_array, edge_order=1)
    v_z_array = np.gradient(z_pos_array, t_array, edge_order=1)
    a_z_array = np.gradient(v_z_array, t_array, edge_order=1)
    a_x_array = np.gradient(v_x_array, t_array, edge_order=1)
    v_total_array = (v_x_array + v_z_array)

    z_apex = np.argmax(z_pos_array)
    
    try:
        start = np.argmax(v_total_array[:z_apex])
        plt.axvline(t_array[start], color='r', linestyle='--')
        print("good start")
    except:
        start = 0
        plt.axvline(t_array[start], color='r', linestyle='--')
        print("bad start")
        
    stop = -1
    plt.scatter(t_array, x_rot_array, label='x_rot')
    plt.scatter(t_array, y_rot_array, label='y_rot')
    plt.scatter(t_array, z_rot_array, label='z_rot')
    plt.scatter(t_array, w_rot_array, label='w_rot')

    for j in range(start, len(z_pos_array) - 1):
        if z_pos_array[j] > 10.4 and z_pos_array[j + 1] <= 10.4:
            stop = j
            print("good stop")
            plt.scatter(t_array, z_pos_array, label='z')
            plt.axvline(t_array[stop], color='r', linestyle='--')
            plt.legend()
            plt.xlim(t_array[start], t_array[stop])
            plt.show()  
            break
        
    if stop == -1:
        print("bad stop")
        plt.scatter(t_array, z_pos_array, label='z')
        plt.axvline(t_array[stop], color='r', linestyle='--')
        plt.legend()
        plt.xlim(t_array[start], t_array[stop])
        plt.show()  

    avg_backspin = np.mean(angvel[start:stop, 1]) / (2 * np.pi)
    backspin_df = pd.concat([backspin_df, pd.DataFrame({'ftid': [ftid], 'avg_backspin': [avg_backspin]})], ignore_index=True)
    
    # Plot angular velocity about y-axis as a function of time|)
    plt.axhline(avg_backspin, label='angular velocity about y-axis (rps)')
    plt.scatter(t_array,angvel[:, 1] /(2*np.pi), label='angular velocity about y-axis (rps)')
    # plt.scatter(t_array, angvel[:, 0]/(2*np.pi), label='angular velocity about x-axis (rps)')
    # plt.scatter(t_array, angvel[:, 2]/(2*np.pi), label='angular velocity about z-axis (rps)')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Angular Velocity (rps)')
    # plt.title(f'Angular Velocity vs Time for ftid {ftid}')
    plt.xlim(t_array[start]-.5, t_array[stop])
    # plt.axvline(t_array[start], color='r', linestyle='--',label='ft start')
    # plt.legend()
    plt.show()
    
backspin_df.to_csv('scottie240backspin.csv', index=False)